In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image
from IPython.display import clear_output
import warnings
import os 
from glob import glob 
import pandas as pd 
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#디렉토리 생성 

In [2]:
!rm -rf cup

In [3]:
!mkdir -p ./cup/

# 이미지 캡쳐 

In [4]:
def take_photo(last_file_path, quality=1.0):

    js = Javascript('''

        async function takePhoto([last_file_path, quality]) {

            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            video.height = 224
            video.width = 224
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            // google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);


            const capture = document.createElement('button');

            capture.textContent = 'Capture';

            div.appendChild(capture);
            
            file_path = last_file_path

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            // canvas.width = 224;
            // canvas.height = 224;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            
            return [canvas.toDataURL('image/jpeg', quality), file_path];
        }
        
    ''')

    display(js)

    data, file_path = eval_js('takePhoto({})'.format([last_file_path, quality]))
    binary = b64decode(data.split(',')[1])
    return binary, file_path


In [5]:
def find_max_index(file_path):
    import re
    file_names = glob(file_path)
    # file_names = ['Picture 03.jpg', '02.jpg', '1.jpg']
    if len(file_names)==0: return 0

    file_indexes = [int(re.sub('\D','',file_name)) for file_name in file_names]
    # file_indexes = [3, 2, 1]

    max_value = max(file_indexes)
    # max_value = 3
    return max_value

max_index = find_max_index('./cup/*')
print(max_index)

0


In [6]:
last_file_path = "./cup"
while(True):
  binary, file_path = take_photo(last_file_path)
  max_index += 1
  file_name = f'{file_path}/image{max_index}.jpg'
  
  with open(file_name,'wb') as f:
    f.write(binary)
#   print(file_name)
  last_file_path = file_path


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [7]:
!tar cvfz cup.tar.gz cup

cup/
cup/image2.jpg
cup/image1.jpg
cup/image3.jpg


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!cp cup.tar.gz /gdrive/MyDrive/tmp/